# TESLA Stock Price Prediciton using Historical Stock Data

## Project Overview

To-Do:
Brief introduction to project, objectives and scope 
Describe problem statement, predicting stock prices, sentiment analysis and dataset used.

Also descriptions underneath each subtitle

## Data Acquisition and Preprocessing
To-Do: Describe sources from which you optain your data (yfinance and Twitter API), detail steps taken to collect and preprocess the data, data cleaning and handling missing values. Justify using only 2020 onwards.

Explain some stock data will have irregular trading days like holidays and stock splits and dividends can artifically change value  before returnign to normal(explain), sorted by yfinance and auto_adjust

- Data cleaning for future datasets (missing values, data type conversion, removing duplicates)

### Importing Libraries

In [521]:
import yfinance as yf
import pandas as pd
import datetime
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.figure_factory as ff
from sklearn.preprocessing import MinMaxScaler
from scipy.spatial import distance
from sklearn.svm import SVR
from sklearn.model_selection import TimeSeriesSplit
import numpy as np

### Acquiring Data

In [522]:
# Set display options to show four decimal places
pd.set_option('display.float_format', '{:.4f}'.format)

# Download Tesla data using yfinance
ticker = 'TSLA'
# Auto_adjust set to true to account for stock splits and dividends
data = yf.download(ticker, auto_adjust=True)

# Convert downloaded data into a Pandas DataFrame
tsla_df = pd.DataFrame(data, columns=['Open', 'High', 'Low', 'Close', 'Volume'])

# Start date for Tesla stock prices
start_date = '2022-01-01'

# End date for Tesla stock prices
end_date = pd.Timestamp(datetime.date.today())
                          
# End date for Tesla stock prices
# Select data from 01/01/2020 onwards
tsla_df = tsla_df.loc[start_date : end_date]

# Columns indicating price
price_columns = ['Open', 'High', 'Low', 'Close']

# Plot colours for Open, High, Low and Close respectively
price_colours = ['#828cf8', '#ef553b', '#00cc96', '#b983f8']

# Colour for volume plots
volume_colour = '#ffa500'

[*********************100%%**********************]  1 of 1 completed


In [523]:
tsla_df.head()

,Open,High,Low,Close,Volume
Date,,,,,
2022-01-03,382.5833,400.3567,378.6800,399.9267,103931400
2022-01-04,396.5167,402.6667,374.3500,383.1967,100248300
2022-01-05,382.2167,390.1133,360.3367,362.7067,80119800
2022-01-06,359.0000,362.6667,340.1667,354.9000,90336600
2022-01-07,360.1233,360.3100,336.6667,342.3200,84164700


## Exploratory Data Analysis

### Key Statistics

In [524]:
tsla_df.describe()

,Open,High,Low,Close,Volume
count,546.0000,546.0000,546.0000,546.0000,546.0000
mean,237.6013,242.9162,231.7449,237.2689,111530430.5989
std,54.7375,55.7923,53.2271,54.3000,40119601.6538
min,103.0000,111.7500,101.8100,108.1000,41864700.0000
25%,195.3150,198.7850,190.9900,194.7700,83452575.0000
50%,238.0533,243.6217,233.3367,238.2967,104645400.0000
75%,274.2183,279.3325,266.5425,273.2458,132615550.0000
max,396.5167,402.6667,378.6800,399.9267,306590600.0000


In [525]:
# Create a list of Box traces for Price columns 
price_box_traces = []

for i, column in enumerate(price_columns):
    price_box_traces.append(go.Box(y=tsla_df[column],
                                   name=column,
                                  marker=dict(color=price_colours[i])))

# Create the figure and add the traces
fig = go.Figure(data=price_box_traces)

# Update the layout
fig.update_layout(
    title='Box Plot of Tesla Stock Prices',
    yaxis_title='Value ($)'
)

# Show the plot
fig.show()

In [526]:
# Create Box trace for Volume
volume_box_trace = go.Box(y=tsla_df['Volume'],
                          name='Volume',
                         marker=dict(color=volume_colour))

# Create the figure and add the trace
fig = go.Figure(data=volume_box_trace)

# Update the layout
fig.update_layout(
    title='Box Plot of Tesla Volume',
    yaxis_title='Volume'
)

# Show the plot
fig.show()

GIVE A DESCRIPTION OF THE ABOVE VISUALISATION (volume generally simial rbut large outliers probably due to news events) compare to price changes

### Distributions

In [527]:
# Fetch data and labels for Price columns
price_data = [tsla_df[column] for column in price_columns]
group_labels = price_columns

# Create the figure and add data
fig = ff.create_distplot(price_data,
                         group_labels,
                         show_hist=False,
                         show_rug=False,
                         colors=price_colours
                        )

# Update the layout
fig.update_layout(
    title='Distribution of Tesla Stock Prices',
    xaxis_title='Value ($)'
    )

# Show the figure
fig.show()

DESCRIBE VALUE WITH DENSITY PLOT

In [528]:
# Fetch data and labels for Volume column
volume_data = [tsla_df['Volume']]
group_labels = ['Volume']

# Create the figure and add data
fig = ff.create_distplot(volume_data,
                         group_labels,
                         show_hist=False,
                         show_rug=False,
                         colors=[volume_colour]
                        )

# Update the layout
fig.update_layout(
    title='Distribution of Tesla Volume',
    xaxis_title='Volume'
    )

# Show the figure
fig.show()

EXPLAIN VISUALISATIONS

### Trends over Time

CHANGE TO SMA
EXPLAIN EMA AND EMA PARAMETERS AND ROLLING STANDARD DEVIATION AND EMA VS SMA BASED ON CLOSE

In [529]:
# Window size of 20 chosen for smooth but responsive EMA curve
# Rougly 20 trading days per month
window = 20

# Calculate the Exponential Weighted Moving Average
tsla_df['EMA'] = tsla_df['Close'].ewm(span=window, min_periods=window, adjust=False).mean()

# Calculate the rolling Standard Deviation
tsla_df['STD'] = tsla_df['Close'].rolling(window=window).std()

# Create the traces for the Candlestick Plot
candlestick_trace = go.Candlestick(x=tsla_df.index,
                                   open=tsla_df['Open'],
                                   high=tsla_df['High'],
                                   low=tsla_df['Low'],
                                   close=tsla_df['Close'],
                                   name='Candlestick')

# Create the trace for the EMA Line Plot
ema_trace = go.Scatter(x=tsla_df.index,
                       y=tsla_df['EMA'],
                       mode='lines',
                       name='EMA',
                       line=dict(color='blue'))

# Create the trace for the Standard Deviation Line Plot
std_trace = go.Scatter(x=tsla_df.index,
                       y=tsla_df['STD'],
                       mode='lines',
                       name='Standard Deviation',
                       line=dict(color='purple'))

# Create subplots to plot Standard Deviation on the same x-axis but 
# a different plot to the Candlestick Chart
fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.05)

# Add the relevant traces to the correct subplot
fig.add_trace(candlestick_trace, row=1, col=1)

fig.add_trace(ema_trace, row=1, col=1)

fig.add_trace(std_trace, row=2, col=1)

# Update the layout
fig.update_yaxes(title_text="Price", row=1, col=1)

fig.update_yaxes(title_text="STD", row=2, col=1, secondary_y=True)

fig.update_layout(title='Candlestick Chart of Tesla Stock with EMA and Rolling Standard Deviation',
                  xaxis_rangeslider_visible=False)

# Show the figure
fig.show()


EXPLAIN VISUALISATION

In [530]:
# Window size of 25 chosen for smooth but responsive SMA curve
window = 25

# Calculate the Simple Moving Average
tsla_df['Volume_SMA'] = tsla_df['Volume'].rolling(window=window).mean()

# Calculate the rolling Standard Deviation
tsla_df['Volume_STD'] = tsla_df['Volume'].rolling(window=window).std()

# Create the trace for the Volume Line Plot
volume_trace = go.Scatter(x=tsla_df.index,
                       y=tsla_df['Volume'],
                       mode='lines',
                       name='Volume',
                       line=dict(color=volume_colour))

# Create the trace for the Volume SMA Line Plot
volume_sma_trace = go.Scatter(x=tsla_df.index,
                            y=tsla_df['Volume_SMA'],
                            mode='lines',
                            name='Simple Moving Average',
                            line=dict(color='blue'))

# Create the trace for the Volume Standard Deviation Line Plot
volume_std_trace = go.Scatter(x=tsla_df.index,
                             y=tsla_df['Volume_STD'],
                             mode='lines',
                             name='Standard Deviation',
                             line=dict(color='purple'))

# Create subplots to plot Standard Deviation on the same x-axis but 
# a different plot to the Volume Line Plot
fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.05)

# Add the relevant traces to the correct subplot
fig.add_trace(volume_trace, row=1, col=1)

fig.add_trace(volume_sma_trace, row=1, col=1)

fig.add_trace(volume_std_trace, row=2, col=1)

# Update the layout
fig.update_yaxes(title_text="Price", row=1, col=1)

fig.update_yaxes(title_text="STD", row=2, col=1, secondary_y=True)

fig.update_layout(title='Tesla Volume Over Time Plot with Simple Moving Average and Rolling Standard Deviation',
                  xaxis_rangeslider_visible=False)

# Show the figure
fig.show()

EXPLAIN VISUALISATIONS
CHANGE SMA TO VMA

### Correlations among Variables

In [531]:
# Define the correlation matrix
correlation_matrix = tsla_df[['Open', 'High', 'Low', 'Close', 'Volume']].corr()

correlation_matrix

,Open,High,Low,Close,Volume
Open,1.0000,0.9966,0.9955,0.9896,-0.5514
High,0.9966,1.0000,0.9961,0.9953,-0.5389
Low,0.9955,0.9961,1.0000,0.9962,-0.5616
Close,0.9896,0.9953,0.9962,1.0000,-0.5478
Volume,-0.5514,-0.5389,-0.5616,-0.5478,1.0000


EXPLAIN CORRELATIONS AND HOW THEY RELATE TO TRAINING (i.e. WHY THEY ARE ALL INCLUDED)

## Feature Engineering

## Model Selection and Training

EXPLAIN SVR AND RBF KERNEL
EXPLAIN WINDOW_SIZE, I.E. HOW MANY DAYS USED TO PREDICT THE FOLLOWING DAY
WINDOW SIZE 5 MEANS USE 5 DAYS OF DATA AND PREDICT THE 6TH DAY
ALSO EXPLAIN TIME SERIES SPLIT
 We split the data into equal parts based on the number of splits
 i.e. if we have 100 data points and n_splits = 5:
 - the first fold will use the first 20 data points as the training set
   and the next 20 data points as the test set
 - the second fold will use the first 40 data points as the training set
   and the next 20 data points as the test set
 - Third fold will use the first 60 data points as the training set and
   the next 20 data points as the test and so forth

In [532]:
# Creates a sliding window dataset. 
# X contains the input features
# y contains the target values
# dates contains the dates used for target variables for plotting

# Parameters:
#   - data (dataframe): the dataframe containing historical stock data
#                       columns: ['Open', 'High', 'Low', 'Close', 'Volume']
#   - window_size:  number of days used to predict the next day's closing price
def create_dataset(data, window_size):
    X, y, dates = [], [], []
    for i in range (len(data) - window_size):
        # Append window_size number of values from all 5 features
        X.append(data.iloc[i : i + window_size, :].values.flatten())
        # Append the close value from the day after the rolling window
        y.append(data.iloc[i + window_size, 3])
        # Append the test dates for plotting purposes
        dates.append(data.index[i + window_size])
    return np.array(X), np.array(y), np.array(dates)

In [533]:
# EXPLAIN THIS 
# FIX SO IT TAKES IN X AND Y
def rbf_kernel(x_i,x_j,sigma2,ell2):
    if x_j is None:
        d = cdist(x_i, x_i)
    else:
        d = cdist(x_i, x_j)
    K = sigma2*np.exp(-np.power(d, 2)/ell2)
    return K

In [534]:
# Calcualtes Mean Absolute Error. This is the average of the absolute
# differences between predicted and actual values. It provides average
# error in the original units.

# Parameters:
#   - y (list): observed (test) values
#   - y_hat (list): predicted values 

def calculate_mae(y, y_hat):
    absolute_difference = [abs(y_hat[i] - y[i]) for i in range(len(y))]
    mae = sum(absolute_difference)/len(y)
    return mae

In [535]:
# Calculates Mean Squared Error. This is the average of the squared 
# differences between predicted and actual values. Penalizes large
# errors.

# Parameters:
#   - y (list): observed (test) values
#   - y_hat (list): predicted values 

def calculate_mse(y, y_hat):
    squared_difference = [(y[i] - y_hat[i]) ** 2 for i in range(len(y))]
    mse = sum(squared_difference)/len(y)
    return mse

In [536]:
# ADD ASSERT SHAPE = SHAPE


window_size = 1

# Create the sliding window dataset
X, y, dates = create_dataset(tsla_df[['Open', 'High', 'Low', 'Close', 'Volume']], window_size)
# Split the X and y dataset into training/test data on a 80/20 split
split = int(0.8 * len(X))
print(len(X))
# Scale the input features
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)
# Split the input features into training and test data
X_train_scaled, X_test_scaled = X[: split], X[split :]
# Split the target values into training and test data
y_train, y_test = y[: split], y[split :]
# Retrieve the dates corresponding to the test data
dates_test = dates[split :]

# Train SVR Model
model = SVR(kernel='rbf')
model.fit(X_train_scaled, y_train)

# Make predictions
predictions = model.predict(X_test_scaled)

# Calculate MAE
mae = calculate_mae(y_test, predictions)
print("Mean Absolute Error: ", round(mae, 2))

# Calculate MSE
mse = calculate_mse(y_test, predictions)
print("Mean Squared Error: ", round(mse, 2))

# Create the trace for the actual values
actual_trace = go.Scatter(
    x=dates_test,
    y=y_test,
    mode='lines',
    name='Actual Values',
    marker=dict(color='#ff6361')
)

# Create the trace for the predicted values
predicted_trace = go.Scatter(
    x=dates_test,
    y=predictions,
    mode='lines',
    name='Predicted Values',
    marker=dict(color='#003f5c')
)

# Create the layout
layout = go.Layout(
    title='Actual vs Predicted Closing Prices',
    xaxis=dict(title='Date'),
    yaxis=dict(title='Closing Price')
)

# Create the figure
fig = go.Figure(data=[actual_trace, predicted_trace], layout=layout)

# Show the plot
fig.show()

545
Mean Absolute Error:  25.58
Mean Squared Error:  1006.55


In [537]:
# Scale the input features
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# Define the number of splits (folds) for time series cross-validation
n_splits = 10
tscv = TimeSeriesSplit(n_splits=n_splits)

# Lists to store evaluation metrics
mae_scores = []
mse_scores = []

# Iterate over time series cross-validation folds
for train_index, test_index in tscv.split(X_scaled):
    # Split the input features into training and test data
    X_train, X_test = X_scaled[train_index], X_scaled[test_index]
    # Split the target values into training and test data
    y_train, y_test = y[train_index], y[test_index]

    # Train SVR Model
    model = SVR(kernel='rbf')
    model.fit(X_train, y_train)

    # Make predictions
    predictions = model.predict(X_test)

    # Calculate MAE per fold
    mae = calculate_mae(y_test, predictions)
    mae_scores.append(mae)

    # Calculate MSE per fold
    mse = calculate_mse(y_test, predictions)
    mse_scores.append(mse)

# Calculate the average MAE from all folds
print("Mean MAE: ", round(np.mean(mae_scores), 2))
# Calculate the average MSE from all folds
print("Mean MSE: ", round(np.mean(mse_scores), 2))


Mean MAE:  27.02
Mean MSE:  1778.66


SVM: Try grid saerch, different kernels, along with justificaiton of window_size and date range
    try other models like lstm and fiddle with window_size, 
 Interpret results, analyse feature importance, model coefficients and insights